In [17]:
# Import dependencies
import hvplot.pandas
import pandas as pd
import requests
import json
import time
from matplotlib import pyplot as plt
from config import geoapify_key


# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")


In [18]:
# Read CSVs into pandas dataframe, verify it read in correctly
health_percent_df = pd.read_csv('Resources/health_as_percent_2020.csv')
country_capital = pd.read_csv('Resources/country-list.csv')


country_capital = country_capital.rename(columns={'country' : 'Country'})

health_as_percent_merged = pd.merge(health_percent_df, country_capital, on="Country", how="left")
#health_as_percent_merged


In [19]:
# Sorting Values to extract top 10 countries with highest health expenditure %
health_as_percent_merged = health_as_percent_merged.sort_values(by='Current health expenditure (% of GDP)', ascending=False)
#health_as_percent_merged

In [20]:
# Several rows do not have data in capital column, drop those for API search
health_as_percent_clean = health_as_percent_merged.dropna(how='any')
health_as_percent_clean.reset_index()

,index,Country,Current health expenditure (% of GDP),capital,type
0,29,United States,18.815826,"Washington, D.C.",countryCapital
1,3,Canada,12.939675,Ottawa,countryCapital
2,8,Germany,12.822489,Berlin,countryCapital
3,7,France,12.205050,Paris,countryCapital
4,26,Switzerland,11.796350,Bern,countryCapital
5,1,Austria,11.474260,Vienna,countryCapital
6,20,Norway,11.417550,Oslo,countryCapital
7,25,Sweden,11.379928,Stockholm,countryCapital
8,18,Netherlands,11.135927,Amsterdam,countryCapital
9,2,Belgium,11.064532,Brussels,countryCapital


In [22]:
health_as_percent_clean = health_as_percent_clean[["Country", "Current health expenditure (% of GDP)", "capital"]]
health_as_percent_clean = health_as_percent_clean.rename(columns={"Current health expenditure (% of GDP)" : "Health Expenditure as % GDP" , "capital" : "Capital"})
health_as_percent_clean.sort_index()
#health_as_percent_clean

,Country,Health Expenditure as % GDP,Capital
0,Australia,10.648995,Canberra
1,Austria,11.474260,Vienna
2,Belgium,11.064532,Brussels
3,Canada,12.939675,Ottawa
5,Denmark,10.528175,Copenhagen
6,Finland,9.613496,Helsinki
7,France,12.205050,Paris
8,Germany,12.822489,Berlin
9,Greece,9.508207,Athens
10,Hungary,7.250314,Budapest


In [6]:
# Filter clean dataframe to first 10 rows (top 10 countries) and columns needed
#health_per_df = health_as_percent_clean.iloc[: 10]
#health_per_df = health_per_df[["Country", "Current health expenditure (% of GDP)", "capital"]]

# Rename columns , sort index
#health_per_df = health_per_df.rename(columns={"Current health expenditure (% of GDP)" : "Health Expenditure as % GDP" , "capital" : "Capital"})
#health_per_df = health_per_df.sort_index()
#health_per_df

In [23]:
# Add columns to store lat and long for each country
health_as_percent_clean["Lat"] = ""
health_as_percent_clean["Lon"] = ""

health_as_percent_clean

,Country,Health Expenditure as % GDP,Capital,Lat,Lon
29,United States,18.815826,"Washington, D.C.",,
3,Canada,12.939675,Ottawa,,
8,Germany,12.822489,Berlin,,
7,France,12.205050,Paris,,
26,Switzerland,11.796350,Bern,,
1,Austria,11.474260,Vienna,,
20,Norway,11.417550,Oslo,,
25,Sweden,11.379928,Stockholm,,
18,Netherlands,11.135927,Amsterdam,,
2,Belgium,11.064532,Brussels,,


In [8]:
# Add columns to store lat and long for each country
#health_per_df["Lat"] = ""
#health_per_df["Lon"] = ""

#health_per_df.head(1)

In [24]:
# Define the API parameters
#target_city = "Washington, D.C."
params = {
    "apiKey" : geoapify_key,
    "format" : "json"
}

# Set base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

In [10]:
#from pprint import pprint
#response = requests.get(base_url, params=params).json()
#pprint(response)


In [25]:
# Print a miessage to follow up the lat/lon search
print("Starting lat/lon search")

for index, row in health_as_percent_clean.iterrows():
    
    # Get the country's name in order to add to params
    country = row["Country"]
    city = row["Capital"] 
    
    
    # Add the country to the params
    params["text"] = f"{city}" + f", {country}"

    # Make API request
    response = requests.get(base_url, params=params).json()

    #print(response)


    # Extract latitude and longitude
    health_as_percent_clean.loc[index,'Lat'] = response["results"][0]["lat"]
    health_as_percent_clean.loc[index,'Lon'] = response["results"][0]["lon"]
    # Log the search results
    print(f"Coordinates for {country} fetched...")


Starting lat/lon search
Coordinates for United States fetched...
Coordinates for Canada fetched...
Coordinates for Germany fetched...
Coordinates for France fetched...
Coordinates for Switzerland fetched...
Coordinates for Austria fetched...
Coordinates for Norway fetched...
Coordinates for Sweden fetched...
Coordinates for Netherlands fetched...
Coordinates for Belgium fetched...
Coordinates for Japan fetched...
Coordinates for Malta fetched...
Coordinates for Spain fetched...
Coordinates for Australia fetched...
Coordinates for Portugal fetched...
Coordinates for Denmark fetched...
Coordinates for Brazil fetched...
Coordinates for New Zealand fetched...
Coordinates for Chile fetched...
Coordinates for Italy fetched...
Coordinates for Finland fetched...
Coordinates for Iceland fetched...
Coordinates for Greece fetched...
Coordinates for Slovenia fetched...
Coordinates for Colombia fetched...
Coordinates for South Africa fetched...
Coordinates for Bulgaria fetched...
Coordinates for Is

In [26]:
# Verify Lat and Lon were added to each country's capital
health_as_percent_clean
#health_as_percent_clean['Lat'] = health_as_percent_clean['Lat'].astype('float')
#health_as_percent_clean['Lon'] = health_as_percent_clean['Lon'].astype('float')

,Country,Health Expenditure as % GDP,Capital,Lat,Lon
29,United States,18.815826,"Washington, D.C.",38.907432,-77.035092
3,Canada,12.939675,Ottawa,45.420878,-75.690111
8,Germany,12.822489,Berlin,52.517037,13.38886
7,France,12.205050,Paris,48.853495,2.348391
26,Switzerland,11.796350,Bern,46.948474,7.452175
1,Austria,11.474260,Vienna,48.208354,16.372504
20,Norway,11.417550,Oslo,59.91333,10.73897
25,Sweden,11.379928,Stockholm,59.325117,18.071093
18,Netherlands,11.135927,Amsterdam,52.37308,4.892453
2,Belgium,11.064532,Brussels,50.846557,4.351697


In [27]:
# Configure map
health_expenditure_map = health_as_percent_clean.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = 24,
    scale = 2,
    color = "Country",
    hover_cols = ["Country", "Health Expenditure as % GDP"]
)

# Display map plot 
health_expenditure_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lon,Lat]   (Country,Health Expenditure as % GDP)